In [19]:
import numpy
import os

import nibabel as nib # nibabel to handle nifti files
import SimpleITK as sitk
import json

import torch

In [20]:
def reduce_label(input_path, output_path):
    volume = sitk.ReadImage(input_path)

    image_data = sitk.GetArrayFromImage(volume)
    #label_map={0:0 , 1:2 , 2:3 , 3:4 , 4:0 , 5:0 , 6:1 , 7:0 , 8:0 , 9:0 ,10:0 , 11:0 , 12:0 , 13:0 , 14:0, 15:0}
    array = torch.tensor(image_data, dtype=torch.int64)  # Convert the array to a PyTorch tensor
    onehot_encoded = torch.nn.functional.one_hot(array, num_classes=16)
    
    label_reduced = torch.zeros_like(array)
    label_reduced[onehot_encoded[:, :, :, 1] == 1] = 2
    label_reduced[onehot_encoded[:, :, :, 2] == 1] = 3
    label_reduced[onehot_encoded[:, :, :, 3] == 1] = 4
    label_reduced[onehot_encoded[:, :, :, 6] == 1] = 1
    label_reduced = label_reduced.numpy()

    #label_unique_values = numpy.unique(label_reduced)
    #print(label_unique_values)


    img_preprocessed = sitk.GetImageFromArray(label_reduced)
    img_preprocessed.CopyInformation(volume)
    
    sitk.WriteImage(img_preprocessed, output_path)


In [21]:
def all_reduce_label(input_json, output_json):
    with open(input_json, 'r') as f:
        info = json.load(f)

    root_dir = os.path.join(os.getcwd(), "..", info["root_dir"])

    for key in ["training", "valid", "test"]:
        for fix_move in ["0", "1"]:
            for index,image_label in enumerate(info[key][fix_move]):
                label = os.path.join(root_dir, image_label["label"])
                label_name = os.path.basename(label)
                label_name = label_name.split('.')[0] + "_reduce.nii.gz"
                label_new = os.path.join(os.path.dirname(label), label_name)

                reduce_label(label, label_new)
                info[key][fix_move][index]["label"] = os.path.relpath(label_new, root_dir)

    with open(output_json, "w") as outfile:
        json.dump(info, outfile, indent=4)

In [22]:
all_reduce_label("../dataset/amos/amos22_dataset.json", "../dataset/amos/amos22_dataset_reduced_label.json")

In [23]:
def change_orientation_volume(input_path, output_path):
    volume = sitk.ReadImage(input_path) # read and cast to float32
    reoriented = sitk.DICOMOrient(volume, 'LAS')
    sitk.WriteImage(reoriented, output_path)

In [24]:
def all_reorient(input_json, output_json):
    with open(input_json, 'r') as f:
        info = json.load(f)

    root_dir = os.path.join(os.getcwd(), "..", info["root_dir"])

    for key in ["training", "valid", "test"]:
        for fix_move in ["0", "1"]:
            for index,image_label in enumerate(info[key][fix_move]):

                image = os.path.join(root_dir, image_label["image"])
                image_name = os.path.basename(image)
                image_name = image_name.split('.')[0] + "_ras.nii.gz"
                image_new = os.path.join(os.path.dirname(image), image_name)

                label = os.path.join(root_dir, image_label["label"])
                label_name = os.path.basename(label)
                label_name = label_name.split('.')[0] + "_ras.nii.gz"
                label_new = os.path.join(os.path.dirname(label), label_name)

                change_orientation_volume(image, image_new)
                change_orientation_volume(label, label_new)
                info[key][fix_move][index]["label"] = os.path.relpath(label_new, root_dir)
                info[key][fix_move][index]["image"] = os.path.relpath(image_new, root_dir)

    with open(output_json, "w") as outfile:
        json.dump(info, outfile, indent=4)

In [25]:
all_reorient("../dataset/amos/amos22_dataset_reduced_label.json", "../dataset/amos/amos22_dataset_reorient.json")